# Strategy

```
+-----+                             +---------+
| Order  | ------------- >   | Promotion  |
+-----+                             +---------+
|           |                              |                  |
+-----+                             +---------+
| total()  |                              | discount()   |
| due()   |                              +---------+ 
+-----+                                        ^
                                                     |
                   +-----------------+-----------------+
                    |                                |                                |
          +-----------+       +------------+            +--------------+
           | FidelityPromo |        | BulkItemPromo |             | LargeOrderPromo |
          +-----------+       +------------+            +--------------+
```

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        '''Return discount as a positive dollar amount'''
            
class FidelityPromo(Promotion):
    '''5% discount for customers with 1000 or more fidelity points'''

    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    '''10% discount for each LineItem with 20 or more units'''
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    '''7% discount for order with 10 or more distinct items'''
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

Customer = namedtuple('Customer', 'name fidelity')

joe = Customer('Joe', 0)
ann = Customer('Ann', 1100)

cart = [LineItem('banana', 4, 0.5),
       LineItem('apple', 10, 1.5),
       LineItem('watermellona', 5, 5.0),]
o1 = Order(joe, cart, FidelityPromo())
print(o1)

banana_cart = [LineItem('banana', 30, 0.5), LineItem('apple', 10, 1.5)]
o2 = Order(joe, banana_cart, BulkItemPromo())
print(o2)

long_cart = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
o3 = Order(joe, long_cart, LargeOrderPromo())
print(o3)

o4 = Order(joe, cart, LargeOrderPromo())
print(o4)

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_promo(order):
    '''5% discount for customers with 1000 or more fidelity points'''
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    '''10% discount for each LineItem with 20 or more units'''
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount


def large_order_promo(order):
    '''7% discount for order with 10 or more distinct items'''
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


Customer = namedtuple('Customer', 'name fidelity')

joe = Customer('Joe', 0)
ann = Customer('Ann', 1100)

cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellona', 5, 5.0), ]
o1 = Order(joe, cart, fidelity_promo)
print(o1)

banana_cart = [LineItem('banana', 30, 0.5), LineItem('apple', 10, 1.5)]
o2 = Order(joe, banana_cart, bulk_item_promo)
print(o2)

long_cart = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
o3 = Order(joe, long_cart, large_order_promo)
print(o3)

o4 = Order(joe, cart, large_order_promo)
print(o4)

---

# Command